In [12]:
import torch

In [116]:
import struct 

first_size = 32 * 32 * 16 * 16
second_size = 16 * 16 * 4 * 4
third_size = 4 * 4

buffer_a, buffer_b, buffer_c = [], [], []

with open('/content/weights1', 'rb') as f:
  buffer_a = struct.unpack('f' * first_size, f.read(4 * first_size))

with open('/content/weights2', 'rb') as f:
  buffer_b = struct.unpack('f' * second_size, f.read(4 * second_size))

with open('/content/weights3', 'rb') as f:
  buffer_c = struct.unpack('f' * third_size, f.read(4 * third_size))

with open('/content/input', 'rb') as f:
  buffer_input = struct.unpack('f' * 1024, f.read(4 * 1024))

In [139]:
tensor_a = torch.Tensor(buffer_a).reshape((16 * 16, 32 * 32))
print(tensor_a.shape)

tensor_b = torch.Tensor(buffer_b).reshape((4 * 4, 16 * 16,))
print(tensor_b.shape)

tensor_c = torch.Tensor(buffer_c).reshape((1, 4 * 4))
print(tensor_c.shape)

torch.Size([256, 1024])
torch.Size([16, 256])
torch.Size([1, 16])


In [173]:
class Net(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = torch.nn.Linear(32 * 32, 16 * 16, bias=False)
    self.fc2 = torch.nn.Linear(16 * 16, 4 * 4, bias=False)
    self.fc3 = torch.nn.Linear(4 * 4, 1, bias=False)

    self.fc1.weight = torch.nn.Parameter(tensor_a, requires_grad=True)
    self.fc2.weight = torch.nn.Parameter(tensor_b, requires_grad=True)
    self.fc3.weight = torch.nn.Parameter(tensor_c, requires_grad=True)

    self.sigmoid = torch.nn.Sigmoid()

  def forward(self, input):
    x = input
    x = self.sigmoid(self.fc1(x))
    x = self.sigmoid(self.fc2(x))
    x = self.sigmoid(self.fc3(x))

    return x


In [174]:
net = Net()

input = torch.Tensor(buffer_input).reshape((1, 1024))
x = net(input)
print(x)

tensor([[0.8019]], grad_fn=<SigmoidBackward0>)
